# Document Classification with Naive Bayes - Lab

## Introduction

In this lecture, you'll practice implementing the Naive Bayes algorithm on your own.

## Objectives

You will be able to:  

* Implement document classification using naive Bayes
* Understand the need for the Laplacian smoothing correction
* Explain how to code a bag of words representation

## Import the Dataset

To start, import the dataset stored in the text file `SMSSpamCollection`.

In [33]:
import pandas as pd
import numpy as np
df = pd.read_csv('SMSSpamCollection', sep='\t', names=['label', 'text'])
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Account for Class Imbalance

To help your algorithm perform more accurately, subset the dataset so that the two classes are of equal size. To do this, keep all of the instances of the minority class (spam) and subset examples of the majority class (ham) to an equal number of examples.

In [34]:
minority = df[df.label=='spam']
undersampled_majority = df[df.label=='ham'].sample(len(minority))
df2 = pd.concat([minority, undersampled_majority])
df2.label.value_counts()


ham     747
spam    747
Name: label, dtype: int64

In [35]:
p_classes = dict(df2.label.value_counts(normalize=True))

## Train - Test Split

Now implement a train test split on your dataset.

In [36]:
# Import sklearn's train_test_split
from sklearn.model_selection import train_test_split

# Define our X (features), and y (target)
y=df2.label
X=df2.text

# Spliting the data 80% train, and 20% test
X_train, X_test, y_train, y_test = train_test_split(X,y,  random_state=22)

# Make the split data into a dataframe
train_df = pd.concat([X_train, y_train], axis=1) 
test_df = pd.concat([X_test, y_test], axis=1)

train_df.head()


,text,label
3092,LORD OF THE RINGS:RETURN OF THE KING in store ...,spam
480,When're you guys getting back? G said you were...,ham
4777,Quite lor. But dun tell him wait he get compla...,ham
2655,Its sarcasm.. .nt scarcasim,ham
1842,I am in your office na.,ham


## Create the word frequency dictionary for each class

Create a word frequency dictionary for each class.

In [37]:
word_freq = {} #Will be a nested dictionary of class_i : {word1:freq, word2:freq..., wordn:freq},.... class_m : {}
labels = train_df.label.unique()
for label in labels:
    temp_df = train_df[train_df.label==label]
    bag = {}
    for row in temp_df.index:
        doc = temp_df['text'][row]
        for word in doc.split():
            bag[word] = bag.get(word, 0) + 1
    word_freq[label] = bag

## Count the Total Corpus Words
Calculate V, the total number of words in the corpus.

In [38]:
vocabulary = set()
for text in train_df.text:
    for word in text.split():
        vocabulary.add(word)
V = len(vocabulary)
V

5877

## Create a Bag of Words Function

Before implementing the entire Naive Bayes algorithm, create a helper function `bag_it()` to create a bag of words representation from a document's text.

In [39]:
def bag_it(email):
    bag = {}
    for word in email.split():
        bag[word]=bag.get(word,0)+1
    return bag

## Implementing Naive Bayes

Now, implement a master function to build a naive Bayes classifier. Be sure to use the logarithmic probabilities to avoid underflow.

In [40]:
def classify_email(email, word_freq, p_classes, V, return_posteriors=False):
    bag = bag_it(doc)
    classes = []
    posteriors = []
    for class_ in word_freq.keys():
        p = np.log(p_classes[class_])
        for word in bag.keys():
            num = bag[word]+1
            denom = word_freq[class_].get(word, 0) + V
            p += np.log(num/denom)
        classes.append(class_)
        posteriors.append(p)
    if return_posteriors:
        print(posteriors)
    return classes[np.argmax(posteriors)]


## Test Out Your Classifier

Finally, test out your classifier and measure its accuracy. Don't be perturbed if your results are sub-par; industry use cases would require substantial additional preprocessing before implementing the algorithm in practice.

In [41]:
y_hat_train = X_train.map(lambda x: classify_email(x, word_freq, p_classes, V))
residuals = y_train == y_hat_train
residuals.value_counts(normalize=True)

True     0.501786
False    0.498214
dtype: float64

## Summary

Well done! In this lab, you practiced implementing naive Bayes' for document classification!